In [1]:
import os, sys
os.getcwd()
#!pip install azure-storage-blob --user
#!pip install storefact --user

'/home/jovyan/openScience/code-data'

In [2]:
import os, sys
import configparser
sys.path.append('/home/jovyan/.local/lib/python3.6/site-packages/')
print(sys.path)

os.path.abspath("AzureDownload/config.txt")
os.getcwd()
config = configparser.ConfigParser()
config.read("/home/jovyan/AzureDownload/config.txt")
config.sections()

['/usr/local/spark/python', '/usr/local/spark/python/lib/py4j-0.10.7-src.zip', '/opt/conda/lib/python36.zip', '/opt/conda/lib/python3.6', '/opt/conda/lib/python3.6/lib-dynload', '', '/home/jovyan/.local/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages/IPython/extensions', '/home/jovyan/.ipython', '/home/jovyan/.local/lib/python3.6/site-packages/']


['configuration', 'credentials']

### Credentials setup, read the WoS jounral name mapped table from Azure

In [3]:
import time
from azure.storage.blob import BlockBlobService

CONTAINERNAME = "mag-2019-01-25"
BLOBNAME= "MAGwosJournalMatch/OpenSci3Journal.csv/part-00000-tid-8679026268804875386-7586e989-d017-4b12-9d5a-53fc6497ec02-1116-c000.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/OpenSci3Journal.csv"

block_blob_service=BlockBlobService(account_name=config.get("configuration","account"),account_key=config.get("configuration","password"))
#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 1.5608294010162354 seconds to download MAGwosJournalMatch/OpenSci3Journal.csv/part-00000-tid-8679026268804875386-7586e989-d017-4b12-9d5a-53fc6497ec02-1116-c000.csv


In [4]:
import pandas as pd

openJ = pd.read_csv('OpenSci3Journal.csv', escapechar='\\', encoding='utf-8')
openJ.count()

Tag                  2945
PaperId              2945
AuthorIdsOrder       2945
AuthorNamesOrder     2945
authorCount          2945
unknownCount         2945
femaleCount          2945
femaleProp           2631
1st                  2114
2nd                  1607
3rd                  1267
4th                   962
5th                   743
last                 2089
FoSNames             2945
Year                 2945
DocType              2945
Journal              2557
Publisher            2360
Doi                  2937
Title                2945
EstimatedCitation    2945
IndexedAbstract      1916
WoSID                1499
WoSjournal           1499
dtype: int64

### To verify that the Spark output is consistent, we compare the pandas dataframes before and after the WoS jounral mapping

In [6]:
open0 = pd.read_csv('OpenSci3.csv', escapechar='\\', encoding='utf-8')
open0.count()

Tag                  2926
PaperId              2926
AuthorIdsOrder       2926
AuthorNamesOrder     2926
authorCount          2926
unknownCount         2926
femaleCount          2926
femaleProp           2614
1st                  2099
2nd                  1597
3rd                  1260
4th                   958
5th                   737
last                 2075
FoSNames             2926
Year                 2926
DocType              2926
Journal              2542
Publisher            2346
Doi                  2918
Title                2926
EstimatedCitation    2926
IndexedAbstract      1906
dtype: int64

### Compare matched MAG journal names and WoS journal names

In [7]:
openJ['Journal'] = openJ.Journal.str.lower()
openJ['WoSjournal'] = openJ.WoSjournal.str.lower()
matched = openJ[openJ['Journal'] == openJ['WoSjournal']]
matched.count()

Tag                  1281
PaperId              1281
AuthorIdsOrder       1281
AuthorNamesOrder     1281
authorCount          1281
unknownCount         1281
femaleCount          1281
femaleProp           1181
1st                   949
2nd                   749
3rd                   610
4th                   482
5th                   381
last                  943
FoSNames             1281
Year                 1281
DocType              1281
Journal              1281
Publisher            1197
Doi                  1281
Title                1281
EstimatedCitation    1281
IndexedAbstract      1001
WoSID                1281
WoSjournal           1281
dtype: int64

### Matching with UCSD map of science journal names

In [8]:
journalMap = pd.read_csv('WoSmatch/journalName.csv')
journalMap['journal_name'] = journalMap.journal_name.str.lower()
JwosMap = journalMap[journalMap['source_type']=="Thomson"]
MAGmatched = pd.merge(openJ, JwosMap, left_on=['Journal'], right_on=['journal_name'], how='left')
MAGmatched.count()

Tag                  2945
PaperId              2945
AuthorIdsOrder       2945
AuthorNamesOrder     2945
authorCount          2945
unknownCount         2945
femaleCount          2945
femaleProp           2631
1st                  2114
2nd                  1607
3rd                  1267
4th                   962
5th                   743
last                 2089
FoSNames             2945
Year                 2945
DocType              2945
Journal              2557
Publisher            2360
Doi                  2937
Title                2945
EstimatedCitation    2945
IndexedAbstract      1916
WoSID                1499
WoSjournal           1499
journ_id             1641
journal_name         1641
source_type          1641
dtype: int64

In [9]:
WoSmatched = pd.merge(openJ, JwosMap, left_on=['WoSjournal'], right_on=['journal_name'], how='left')
WoSmatched.count()

Tag                  2945
PaperId              2945
AuthorIdsOrder       2945
AuthorNamesOrder     2945
authorCount          2945
unknownCount         2945
femaleCount          2945
femaleProp           2631
1st                  2114
2nd                  1607
3rd                  1267
4th                   962
5th                   743
last                 2089
FoSNames             2945
Year                 2945
DocType              2945
Journal              2557
Publisher            2360
Doi                  2937
Title                2945
EstimatedCitation    2945
IndexedAbstract      1916
WoSID                1499
WoSjournal           1499
journ_id             1150
journal_name         1150
source_type          1150
dtype: int64

### Combining matched journal names from WoS and MAG to the UCSD map of science

In [10]:
MAGmatched.update(WoSmatched)
MAGmatched.count()

Tag                  2945
PaperId              2945
AuthorIdsOrder       2945
AuthorNamesOrder     2945
authorCount          2945
unknownCount         2945
femaleCount          2945
femaleProp           2631
1st                  2114
2nd                  1607
3rd                  1267
4th                   962
5th                   743
last                 2089
FoSNames             2945
Year                 2945
DocType              2945
Journal              2557
Publisher            2360
Doi                  2937
Title                2945
EstimatedCitation    2945
IndexedAbstract      1916
WoSID                1499
WoSjournal           1499
journ_id             1702
journal_name         1702
source_type          1702
dtype: int64

### Mapping from matched jounrals to subdisciplines

In [11]:
JsubMap = pd.read_csv('WoSmatch/jounral-subdiscipline.csv')
JsubMap.journ_id = JsubMap.journ_id.astype('float64')
subMatched = pd.merge(MAGmatched, JsubMap, left_on=['journ_id'], right_on=['journ_id'], how='left').drop(columns='formal_name')
subMatched.count()
#subMatched.dtypes

Tag                  9419
PaperId              9419
AuthorIdsOrder       9419
AuthorNamesOrder     9419
authorCount          9419
unknownCount         9419
femaleCount          9419
femaleProp           8711
1st                  8020
2nd                  3297
3rd                  2432
4th                  1896
5th                  1123
last                 7961
FoSNames             9419
Year                 9419
DocType              9419
Journal              9031
Publisher            7419
Doi                  9411
Title                9419
EstimatedCitation    9419
IndexedAbstract      5181
WoSID                3292
WoSjournal           3292
journ_id             8176
journal_name         8176
source_type          8176
jfraction            8176
subd_id              8176
dtype: int64

In [12]:
subTable = pd.read_csv('WoSmatch/subdiscipline.csv')
subTable.subd_id = subTable.subd_id.astype('float64')
subNameMatched = pd.merge(subMatched, subTable, left_on=['subd_id'], right_on=['subd_id'], how='left').drop(columns=['size','x','y'])
subNameMatched.count()

Tag                  9419
PaperId              9419
AuthorIdsOrder       9419
AuthorNamesOrder     9419
authorCount          9419
unknownCount         9419
femaleCount          9419
femaleProp           8711
1st                  8020
2nd                  3297
3rd                  2432
4th                  1896
5th                  1123
last                 7961
FoSNames             9419
Year                 9419
DocType              9419
Journal              9031
Publisher            7419
Doi                  9411
Title                9419
EstimatedCitation    9419
IndexedAbstract      5181
WoSID                3292
WoSjournal           3292
journ_id             8176
journal_name         8176
source_type          8176
jfraction            8176
subd_id              8176
subd_name            8176
disc_id              8176
dtype: int64

### Since each journal has a distribution of corresponding disciplines, we will collect the disipline vectors into new columns

In [14]:
import numpy as np
majTable = pd.read_csv('WoSmatch/discipline.csv')
majTable.disc_id = majTable.disc_id.astype('float64')
discMatched = pd.merge(subNameMatched, majTable, left_on=['disc_id'], right_on=['disc_id'], how='left').drop(columns=['color','x','y'])
discMatched.jfraction = discMatched.jfraction.astype('str')
discMatched.subd_name = discMatched.subd_name.astype('str')
discMatched.disc_name = discMatched.disc_name.astype('str')
temp = pd.DataFrame() 
temp = discMatched[['PaperId','WoSID','WoSjournal','jfraction','subd_name','disc_name']]
temp['jfraction'] = discMatched.groupby(['PaperId'])['jfraction'].transform(lambda x: ';'.join(x)).replace('nan', np.nan)
temp['subd_name'] = discMatched.groupby(['PaperId'])['subd_name'].transform(lambda x: ';'.join(x)).replace('nan', np.nan)
temp['disc_name'] = discMatched.groupby(['PaperId'])['disc_name'].transform(lambda x: ';'.join(x)).replace('nan', np.nan)
temp2 = temp.drop_duplicates()
temp2.count()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

PaperId       2945
WoSID         1499
WoSjournal    1499
jfraction     1702
subd_name     1702
disc_name     1702
dtype: int64

### Groupby number matches and we merge the mapped discipline data back to the OpenSci3.csv

In [15]:
OpenSci3Disc = pd.merge(open0, temp2, left_on=['PaperId'], right_on=['PaperId'], how='left')
OpenSci3Disc.count()

Tag                  2926
PaperId              2926
AuthorIdsOrder       2926
AuthorNamesOrder     2926
authorCount          2926
unknownCount         2926
femaleCount          2926
femaleProp           2614
1st                  2099
2nd                  1597
3rd                  1260
4th                   958
5th                   737
last                 2075
FoSNames             2926
Year                 2926
DocType              2926
Journal              2542
Publisher            2346
Doi                  2918
Title                2926
EstimatedCitation    2926
IndexedAbstract      1906
WoSID                1490
WoSjournal           1490
jfraction            1694
subd_name            1694
disc_name            1694
dtype: int64

In [16]:
OpenSci3Disc.to_csv('OpenSci3Discipline.csv',index=False, sep=',', encoding='utf-8')